In [93]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
# from google.colab import data_table
# data_table.enable_dataframe_formatter()
import warnings
warnings.filterwarnings('ignore')
from pycaret.classification import *
def dia():
    from datetime import date, timedelta
    dia = date.today() + timedelta(0)
    return dia

def drop_reset_index(df):
    df = df.dropna()
    df = df.reset_index(drop=True)
    df.index += 1
    return df

def grafico(df, nome):
    df = df.reset_index(drop=True)
    df.index += 1
    df['Profit_acu'] = df.Profit.cumsum()
    profit = round(df.Profit_acu.tail(1).item(),2)
    ROI = round((df.Profit_acu.tail(1)/len(df)*100).item(),2)
    df.Profit_acu.plot(title=nome, xlabel='Entradas', ylabel='Stakes')
    print("Profit:",profit,"stakes em", len(df),"jogos")
    print("ROI:",ROI,"%")

In [94]:
jogos_do_dia = pd.read_csv('C:/Users/thiag/OneDrive/curso python/modelo_leandro_2/flashscore/Jogos_do_Dia/'+str(dia())+'_Jogos_do_Dia.csv')
jogos_do_dia = jogos_do_dia[['League','Round','Date','Time','Home','Away','FT_Odd_H','FT_Odd_D','FT_Odd_A','FT_Odd_Over25','FT_Odd_Under25','FT_Odd_BTTS_Yes','FT_Odd_BTTS_No']]
Jogos_do_Dia = drop_reset_index(jogos_do_dia)
#Jogos_do_Dia

In [95]:
#base = pd.read_csv("https://github.com/futpythontrader/YouTube/blob/main/Base_de_Dados/futpythontraderpunter.csv?raw=true")

base= pd.read_excel("C:/Users/thiag/OneDrive/curso python/modelo_leandro_2/Base_FlahScore.xlsx")
base = base[base['Season'].isin(['2022','2023', '2022/2023']) == True]

base = base[['Date','Time','League','Season','Round','Home','Away','HT_Goals_H','HT_Goals_A','FT_Goals_H','FT_Goals_A','Odd_H',
'Odd_D','Odd_A','Odd_Over25','Odd_Under25','Odd_BTTS_Yes','Odd_BTTS_No']]

base.columns = ['Date', 'Time', 'League', 'Season', 'Round', 'Home', 'Away', 'HT_Goals_H', 'HT_Goals_A', 
                'FT_Goals_H', 'FT_Goals_A', 'FT_Odd_H', 'FT_Odd_D', 'FT_Odd_A', 'FT_Odd_Over25', 'FT_Odd_Under25', 
                'FT_Odd_BTTS_Yes','FT_Odd_BTTS_No']

In [96]:
# Probabilidades
base['p_H'] = 1 / base.FT_Odd_H
base['p_D'] = 1 / base.FT_Odd_D
base['p_A'] = 1 / base.FT_Odd_A

# CV das Odds do Match Odds
base['CV_HDA'] = base[['p_H','p_D','p_A']].std(ddof=0, axis=1) / base[['p_H','p_D','p_A']].mean(axis=1)

# Pontos
base.loc[base['FT_Goals_H'] > base['FT_Goals_A'], 'Ptos_H'] = 3
base.loc[base['FT_Goals_H'] == base['FT_Goals_A'], 'Ptos_H'] = 1
base.loc[base['FT_Goals_H'] < base['FT_Goals_A'], 'Ptos_H'] = 0

base.loc[base['FT_Goals_H'] < base['FT_Goals_A'], 'Ptos_A'] = 3
base.loc[base['FT_Goals_H'] == base['FT_Goals_A'], 'Ptos_A'] = 1
base.loc[base['FT_Goals_H'] > base['FT_Goals_A'], 'Ptos_A'] = 0

base["Ptos_H"] = base["Ptos_H"].astype('int64')
base["Ptos_A"] = base["Ptos_A"].astype('int64')

base['Media_Ptos_H'] = base.groupby('Home')['Ptos_H'].rolling(window=10, min_periods=2).mean().reset_index(0,drop=True)
base['Media_Ptos_A'] = base.groupby('Away')['Ptos_A'].rolling(window=10, min_periods=2).mean().reset_index(0,drop=True)

base['DesvPad_Ptos_H'] = base.groupby('Home')['Ptos_H'].rolling(window=10, min_periods=2).std(ddof=0).reset_index(0,drop=True)
base['DesvPad_Ptos_A'] = base.groupby('Away')['Ptos_A'].rolling(window=10, min_periods=2).std(ddof=0).reset_index(0,drop=True)

base['CV_Ptos_H'] = base['DesvPad_Ptos_H'] / base['Media_Ptos_H']
base['CV_Ptos_A'] = base['DesvPad_Ptos_A'] / base['Media_Ptos_A']

# Valor do Gol
base['VG_H'] = base.FT_Goals_H * base.p_A
base['VG_A'] = base.FT_Goals_A * base.p_H

base['Media_VG_H'] = base.groupby('Home')['VG_H'].rolling(window=10, min_periods=2).mean().reset_index(0,drop=True)
base['Media_VG_A'] = base.groupby('Away')['VG_A'].rolling(window=10, min_periods=2).mean().reset_index(0,drop=True)

base['DesvPad_VG_H'] = base.groupby('Home')['VG_H'].rolling(window=10, min_periods=2).std(ddof=0).reset_index(0,drop=True)
base['DesvPad_VG_A'] = base.groupby('Away')['VG_A'].rolling(window=10, min_periods=2).std(ddof=0).reset_index(0,drop=True)

base['CV_VG_H'] = base['DesvPad_VG_H'] / base['Media_VG_H']
base['CV_VG_A'] = base['DesvPad_VG_A'] / base['Media_VG_A']

# Custo do Gol
base['CG_H'] = base.p_H / base.FT_Goals_H
base['CG_A'] = base.p_A / base.FT_Goals_A

base['Media_CG_H'] = base.groupby('Home')['CG_H'].rolling(window=10, min_periods=2).mean().reset_index(0,drop=True)
base['Media_CG_A'] = base.groupby('Away')['CG_A'].rolling(window=10, min_periods=2).mean().reset_index(0,drop=True)


base['DesvPad_CG_H'] = base.groupby('Home')['CG_H'].rolling(window=10, min_periods=2).std(ddof=0).reset_index(0,drop=True)
base['DesvPad_CG_A'] = base.groupby('Away')['CG_A'].rolling(window=10, min_periods=2).std(ddof=0).reset_index(0,drop=True)

base['CV_CG_H'] = base['DesvPad_CG_H'] / base['Media_CG_H']
base['CV_CG_A'] = base['DesvPad_CG_A'] / base['Media_CG_A']

base['coeficiente_pontos'] = base['Media_Ptos_H'] - base['Media_Ptos_A']
base['cv_pontos'] = base['DesvPad_Ptos_H'] + base['DesvPad_Ptos_A']

base.dropna(inplace=True)
base.replace(np.inf, 1, inplace=True)
base = drop_reset_index(base)
print('pronto')

pronto


In [97]:
base = base.drop(['VG_H', 'VG_A', 'CG_H', 'CG_A'], axis=1)

base.replace(np.inf, 1, inplace=True)
base.dropna(inplace=True)
base.reset_index(inplace=True, drop=True)
base.index = base.index.set_names(['Nº'])
base = base.rename(index=lambda x: x + 1)

base_H = base[['Home','Media_VG_H','DesvPad_VG_H','CV_VG_H','Media_CG_H','DesvPad_CG_H','CV_CG_H', 'Media_Ptos_H', 'DesvPad_Ptos_H']]

base_A = base[['Away','Media_VG_A','DesvPad_VG_A','CV_VG_A','Media_CG_A', 'DesvPad_CG_A','CV_CG_A', 'Media_Ptos_A', 'DesvPad_Ptos_A']]

In [98]:
lista=[]

for a,b,c,d,e,f,g,h,i,j,k,l in zip(Jogos_do_Dia.League,
                                   Jogos_do_Dia.Date,
                                   Jogos_do_Dia.Time,
                                   Jogos_do_Dia.Home,
                                   Jogos_do_Dia.Away,
                                   Jogos_do_Dia.FT_Odd_H,
                                   Jogos_do_Dia.FT_Odd_D,
                                   Jogos_do_Dia.FT_Odd_A,
                                   Jogos_do_Dia.FT_Odd_Over25,
                                   Jogos_do_Dia.FT_Odd_Under25,
                                   Jogos_do_Dia.FT_Odd_BTTS_Yes,
                                   Jogos_do_Dia.FT_Odd_BTTS_No):
    
        League = a
        Date = b
        Time = c
        home = d
        away = e
        FT_Odd_H = f
        FT_Odd_D = g
        FT_Odd_A = h
        FT_Odd_Over25 = i
        FT_Odd_Under25 = j
        FT_Odd_BTTS_Yes = k
        FT_Odd_BTTS_No = l

        df1 = base_H[base_H.Home == home].tail(1)

        df2 = base_A[base_A.Away == away].tail(1)

        jogo = {}

        jogo["League"] = League
        jogo["Date"] = Date
        jogo["Time"] = Time
        jogo["Home"] = home
        jogo["Away"] = away
        jogo["FT_Odd_H"] = FT_Odd_H
        jogo["FT_Odd_D"] = FT_Odd_D
        jogo["FT_Odd_A"] = FT_Odd_A
        jogo["FT_Odd_Over25"] = FT_Odd_Over25
        jogo["FT_Odd_Under25"] = FT_Odd_Under25
        jogo["FT_Odd_BTTS_Yes"] = FT_Odd_BTTS_Yes
        jogo["FT_Odd_BTTS_No"] = FT_Odd_BTTS_No

        try:
            jogo['Media_VG_H'] = df1[df1.Home == home]['Media_VG_H'].iloc[0]
            jogo['DesvPad_VG_H'] = df1[df1.Home == home]['DesvPad_VG_H'].iloc[0]
            jogo['CV_VG_H'] = df1[df1.Home == home]['CV_VG_H'].iloc[0]
            
            jogo['Media_Ptos_H'] = df1[df1.Home == home]['Media_Ptos_H'].iloc[0]
            jogo['DesvPad_Ptos_H'] = df1[df1.Home == home]['DesvPad_Ptos_H'].iloc[0]
           
            jogo['Media_CG_H'] = df1[df1.Home == home]['Media_CG_H'].iloc[0]
            jogo['DesvPad_CG_H'] = df1[df1.Home == home]['DesvPad_CG_H'].iloc[0]
            jogo['CV_CG_H'] = df1[df1.Home == home]['CV_CG_H'].iloc[0]

            jogo['Media_VG_A'] = df2[df2.Away == away]['Media_VG_A'].iloc[0]
            jogo['DesvPad_VG_A'] = df2[df2.Away == away]['DesvPad_VG_A'].iloc[0]
            
            jogo['CV_VG_A'] = df2[df2.Away == away]['CV_VG_A'].iloc[0]
            jogo['Media_Ptos_A'] = df2[df2.Away == away]['Media_Ptos_A'].iloc[0]
            
            jogo['DesvPad_Ptos_A'] = df2[df2.Away == away]['DesvPad_Ptos_A'].iloc[0]
            jogo['Media_CG_A'] = df2[df2.Away == away]['Media_CG_A'].iloc[0]
            
            jogo['DesvPad_CG_A'] = df2[df2.Away == away]['DesvPad_CG_A'].iloc[0]
            jogo['CV_CG_A'] = df2[df2.Away == away]['CV_CG_A'].iloc[0]
            
    
           
        except:
            pass

        lista.append(jogo)


df = pd.DataFrame(lista)
df.reset_index(inplace=True, drop=True)
df.index = df.index.set_names(['Nº'])
df = df.rename(index=lambda x: x + 1)
################################################################################
# Probabilidades
################################################################################
df['p_H'] = 1 / df.FT_Odd_H
df['p_D'] = 1 / df.FT_Odd_D
df['p_A'] = 1 / df.FT_Odd_A
df['p_Over'] = 1 / df.FT_Odd_Over25
df['p_Under'] = 1 / df.FT_Odd_Under25

################################################################################
# CV das Odds do Match Odds
################################################################################
df['CV_HDA'] = df[['p_H','p_D','p_A']].std(ddof=0, axis=1) / df[['p_H','p_D','p_A']].mean(axis=1)

df['coeficiente_pontos'] = df['Media_Ptos_H'] - df['Media_Ptos_A']
df['cv_pontos'] = df['DesvPad_Ptos_H'] + df['DesvPad_Ptos_A']


In [99]:
df

,League,Date,Time,Home,Away,FT_Odd_H,FT_Odd_D,FT_Odd_A,FT_Odd_Over25,FT_Odd_Under25,...,DesvPad_CG_A,CV_CG_A,p_H,p_D,p_A,p_Over,p_Under,CV_HDA,coeficiente_pontos,cv_pontos
Nº,,,,,,,,,,,,,,,,,,,,,
1,ARGENTINA - LIGA PROFESIONAL,10/07/2023,14:00,Barracas Central,Argentinos Jrs,3.75,3.10,2.15,2.50,1.50,...,0.066611,0.219104,0.266667,0.322581,0.465116,0.400000,0.666667,0.237726,0.600000,2.497999
2,ARGENTINA - LIGA PROFESIONAL,10/07/2023,16:30,Colon Santa Fe,Belgrano,2.30,3.00,3.50,2.50,1.50,...,0.095058,0.435899,0.434783,0.333333,0.285714,0.400000,0.666667,0.176970,0.600000,2.399468
3,ARGENTINA - LIGA PROFESIONAL,10/07/2023,19:30,Boca Juniors,Huracan,1.91,3.00,5.00,2.88,1.40,...,0.026227,0.091850,0.523560,0.333333,0.200000,0.347222,0.714286,0.376874,1.500000,1.745362
4,ARGENTINA - LIGA PROFESIONAL,10/07/2023,21:30,Talleres Cordoba,Union de Santa Fe,1.62,3.75,6.00,2.10,1.70,...,0.097683,0.360857,0.617284,0.266667,0.166667,0.476190,0.588235,0.551719,1.100000,2.400693
5,BOLIVIA - DIVISION PROFESIONAL,10/07/2023,21:00,Nacional Potosi,Oriente Petrolero,1.33,5.00,10.00,1.53,2.40,...,0.114066,0.545815,0.751880,0.200000,0.100000,0.653595,0.416667,0.817539,1.900000,1.771780
6,BRAZIL - SERIE B,10/07/2023,20:00,Vila Nova FC,Vitoria,1.87,3.38,4.49,2.41,1.58,...,0.156066,0.735105,0.534759,0.295858,0.222717,0.414938,0.632911,0.379502,0.900000,2.201038
7,EGYPT - PREMIER LEAGUE,10/07/2023,14:30,Ceramica Cleopatra,Pyramids,3.61,3.06,2.09,2.32,1.61,...,0.140999,0.334036,0.277008,0.326797,0.478469,0.431034,0.621118,0.237500,-1.000000,2.179812
8,EGYPT - PREMIER LEAGUE,10/07/2023,14:30,El Daklyeh,El Ismaily,2.82,2.90,2.67,2.50,1.53,...,0.070613,0.389193,0.354610,0.344828,0.374532,0.400000,0.653595,0.034526,-0.200000,1.874693
9,EGYPT - PREMIER LEAGUE,10/07/2023,14:30,El Gaish,Aswan SC,1.96,3.04,4.61,2.40,1.48,...,0.046288,0.233993,0.510204,0.328947,0.216920,0.416667,0.675676,0.343271,0.100000,2.231227


liga = 'ARGENTINA - LIGA PROFESIONAL'

flt = df.League == liga
df = df[flt]
df = drop_reset_index(df)
df

# Carregando o modelo salvo
saved_final_model = load_model('LAYAWAY_ARKAD_ARGENTINA - LIGA PROFESIONAL')

# Fazendo previsões com os novos dados
new_predictions = predict_model(saved_final_model, data=df)
print()
display(new_predictions)
#new_predictions


In [100]:
flt = df.League == 'ARGENTINA - LIGA PROFESIONAL'
df = df[flt]
df = drop_reset_index(df)
saved_final_lr = load_model('LAYAWAY_ARKAD_ARGENTINA - LIGA PROFESIONAL')
new_predictions = predict_model(saved_final_lr, data=df)
new_predictions

Transformation Pipeline and Model Successfully Loaded


,p_H,p_D,p_A,CV_VG_H,CV_VG_A,coeficiente_pontos,cv_pontos,CV_HDA,prediction_label,prediction_score
1,0.266667,0.322581,0.465116,0.900543,1.616780,0.6,2.497999,0.237726,0,0.6622
2,0.434783,0.333333,0.285714,0.639022,1.465590,0.6,2.399468,0.176970,1,0.7617
3,0.523560,0.333333,0.200000,0.807847,1.532307,1.5,1.745362,0.376874,1,0.8630
4,0.617284,0.266667,0.166667,0.890123,0.875426,1.1,2.400693,0.551719,1,0.9431
